<a href="https://colab.research.google.com/github/aditya-unni/SET1/blob/main/webscrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
print("gay")
url = 'https://www.tripadvisor.in/AttractionProductReview-g304554-d16731994-Mumbai_Temple_Tour_in_Private_Vehicle-Mumbai_Maharashtra.html'
response = requests.get(url)
print("gay1")
if response.status_code == 200:
    # Proceed with parsing the HTML content
    print("gay1")
    page_content = response.text
else:
    print(f"Request failed with status code: {response.status_code}")


Hello world


In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page_content, 'html.parser')
# Use soup.select or soup.find to locate specific elements
reviews = soup.select('.yCeTE')  # Example selector for review text


In [ ]:
for review in reviews:
    review_text = review.get_text()
    print(review_text)


In [ ]:
# Fullproof code to collect reviews and user data from TripAdvisor

import sys
import csv
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchElementException
import time

#User Details
profiles_list=[]
combined_data_list = []

# default path to file to store data
path_to_file = "C:\\Users\\vishn\\Desktop\\SET1\\reviews.csv"

# default number of scraped pages
# num_page = 700
num_page = 2

# default tripadvisor website of hotel or things to do (attraction/monument)
# url = "https://www.tripadvisor.in/AttractionProductReview-g304554-d16796488-Private_Full_Day_Mumbai_Sightseeing_Tour_with_Dharavi_Slums-Mumbai_Maharashtra.html"
url = "https://www.tripadvisor.in/Attraction_Review-g304554-d321437-Reviews-Marine_Drive-Mumbai_Maharashtra.html"

# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

# import the webdriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)

# open the file to save the review
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)

column_headings = ["Date", "Title", "Review", "Name", "Username", "Loaction", "Joined_Date"]
csvWriter.writerow(column_headings)

# change the value inside the range to save more or less reviews
for i in range(0, num_page):

    # expand the review
    time.sleep(2)

    container = driver.find_elements("xpath","//div[contains(@data-automation, 'reviewCard')]")
    dates = driver.find_elements("xpath","//div[@class='biGQs _P pZUbB ncFvv osNWb']")

    for j in range(len(container)):
        button = container[j].find_element("xpath", ".//button//span[text()='Read more']/..")
        if button.is_displayed() and button.is_enabled():
            button.click()
        title = container[j].find_element("xpath",".//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']/a/span").text
        review = container[j].find_element("xpath",".//div[@class='biGQs _P pZUbB KxBGd']/span[@class='yCeTE']").text.replace("\n", "  ")

        profile_link = container[j].find_element("xpath", ".//a[contains(@class, 'BMQDV')]").get_attribute("href")
        profiles_list.append(profile_link)

        if len(dates) > j:
            date = " ".join(dates[j].text.split(" ")[-2:])
        else:
            date = "Date Not Available"

#         date = " ".join(dates[j].text.split(" ")[-2:])
#         csvWriter.writerow([date, title, review])
        combined_data = {
            "Date": date,
            "Title": title,
            "Review": review,
            "Name": "",
            "Username": "",
            "Location": "",
            "Joined_Date": ""
        }
        combined_data_list.append(combined_data)

    # change the page
    try:
        nextbutton=driver.find_element("xpath",".//a[contains(@data-smoke-attr,'pagination-next-arrow')]")
        if nextbutton.is_displayed() and nextbutton.is_enabled():
            nextbutton.click()
        else:
            print("Reached the last page.")
            break
    except NoSuchElementException:
        print("Pagination element not found. Reached the last page.")
        break


iteration = 1

for profile_url in profiles_list:

    driver.get(profile_url)
    time.sleep(2)

    intro = driver.find_element("xpath", "//div[contains(@class, 'Me Nb MD NC')]")

    try:
        loc = intro.find_element("xpath", ".//span[@class='PacFI _R S4 H3 LXUOn default']").text
    except NoSuchElementException:
        loc = "Location Not Available"

    try:
        joined = intro.find_element("xpath", ".//span[@class='ECVao _R H3']").text
    except NoSuchElementException:
        joined = "Joined Date Not Available"

    user_info = driver.find_element("xpath", "//span[@class='ecLBS _R shSnD']")

    try:
        username = user_info.find_element("xpath", ".//span[@class='Dsdjn _R']").text
    except NoSuchElementException:
        username = "Username Not Available"

    try:
        name = user_info.find_elements("xpath", ".//span[@class='JWmxy']/h1/span[@class='OUDwj b brsfY']")[0].text
    except (IndexError, NoSuchElementException):
        name = "Name Not Available"

#     print("For iteration ", iteration, ": ", "name, username, joined, loc: \n", name, "\n", username, "\n", joined, "\n", loc, "\n")
#     csvWriter.writerow(["", "", "", name, username, loc, joined])
    combined_data = combined_data_list[iteration - 1]
    combined_data["Name"] = name
    combined_data["Username"] = username
    combined_data["Location"] = loc
    combined_data["Joined_Date"] = joined

    iteration+=1

for data in combined_data_list:
    csvWriter.writerow([data["Date"], data["Title"], data["Review"], data["Name"], data["Username"], data["Location"], data["Joined_Date"]])

driver.quit()

In [ ]:
# Fullproof code to collect reviews and user data from TripAdvisor of multiplePlaces

import sys
import csv
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import multiprocessing


def scrape_and_append_reviews(url, destination_code):
    #User Details
    profiles_list=[]
    combined_data_list = []

    # default number of scraped pages
    num_page = 50

    # default path to file to store data
    path_to_file = "C:\\Users\\vishn\\Desktop\\SET1\\reviews_" + destination_code + ".csv"

    # open the file to save the review
    csvFile = open(path_to_file, 'a', encoding="utf-8")
    csvWriter = csv.writer(csvFile)

    column_headings = ["Date", "Title", "Review", "Destination_ID", "Name", "Username", "Location", "Joined_Date"]
    csvWriter.writerow(column_headings)

    # import the webdriver
    driver = webdriver.Chrome(options=webdriver.ChromeOptions().add_argument('--headless'))
    driver.maximize_window()
    driver.get(url)

    # change the value inside the range to save more or less reviews
    for i in range(0, num_page):

        # expand the review
        time.sleep(2)

        container = driver.find_elements("xpath","//div[contains(@data-automation, 'reviewCard')]")
        dates = driver.find_elements("xpath","//div[@class='biGQs _P pZUbB ncFvv osNWb']")

        for j in range(len(container)):
            button = container[j].find_element("xpath", ".//button//span[text()='Read more']/..")
            if button.is_displayed() and button.is_enabled():
                button.click()
            title = container[j].find_element("xpath",".//div[@class='biGQs _P fiohW qWPrE ncFvv fOtGX']/a/span").text
            review = container[j].find_element("xpath",".//div[@class='biGQs _P pZUbB KxBGd']/span[@class='yCeTE']").text.replace("\n", "  ")

            profile_link = container[j].find_element("xpath", ".//a[contains(@class, 'BMQDV')]").get_attribute("href")
            profiles_list.append(profile_link)

            if len(dates) > j:
                date = " ".join(dates[j].text.split(" ")[-2:])
            else:
                date = "Date Not Available"

            combined_data = {
                "Date": date,
                "Title": title,
                "Review": review,
                "Destination_ID": destination_code,
                "Name": "",
                "Username": "",
                "Location": "",
                "Joined_Date": ""

            }
            combined_data_list.append(combined_data)


        # change the page
        try:
            nextbutton=driver.find_element("xpath",".//a[contains(@data-smoke-attr,'pagination-next-arrow')]")
            if nextbutton.is_displayed() and nextbutton.is_enabled():
                nextbutton.click()
            else:
                print("Reached the last page.")
                break
        except NoSuchElementException:
            print("Pagination element not found. Reached the last page.")
            break


    iteration = 0

    for profile_url in profiles_list:

        driver.get(profile_url)
        time.sleep(2)

        intro = driver.find_element("xpath", "//div[contains(@class, 'Me Nb MD NC')]")

        try:
            loc = intro.find_element("xpath", ".//span[@class='PacFI _R S4 H3 LXUOn default']").text
        except NoSuchElementException:
            loc = "Location Not Available"

        try:
            joined = intro.find_element("xpath", ".//span[@class='ECVao _R H3']").text
        except NoSuchElementException:
            joined = "Joined Date Not Available"

        user_info = driver.find_element("xpath", "//span[@class='ecLBS _R shSnD']")

        try:
            username = user_info.find_element("xpath", ".//span[@class='Dsdjn _R']").text
        except NoSuchElementException:
            username = "Username Not Available"

        try:
            name = user_info.find_elements("xpath", ".//span[@class='JWmxy']/h1/span[@class='OUDwj b brsfY']")[0].text
        except (IndexError, NoSuchElementException):
            name = "Name Not Available"

        combined_data = combined_data_list[iteration - 1]
        combined_data["Name"] = name
        combined_data["Username"] = username
        combined_data["Location"] = loc
        combined_data["Joined_Date"] = joined

        iteration+=1

    for data in combined_data_list:
        csvWriter.writerow([data["Date"], data["Title"], data["Review"], data["Destination_ID"], data["Name"], data["Username"], data["Location"], data["Joined_Date"]])

    driver.quit()

scrape_and_append_reviews("https://www.tripadvisor.in/Attraction_Review-g304554-d311661-Reviews-Colaba-Mumbai_Maharashtra.html", "COL")

In [ ]:
import sys
import csv
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import multiprocessing
from scrapper import scrape_and_append_reviews

code_to_name = {
    "GWOI": "Gateway Of India",
    "MD": "Marine Drive",
    "SHSI" : "Shree Siddhivinayak",
    "BWS" : "Bandra-Worli Sea Link",
    "CST" : "Chhatrapati Shivaji Terminus",
    "KZM" : "KidZania Mumbai",
    "MBGM" : "Mani Bhavan Gandhi Museum",
    "COL" : "Colaba",
    "ELC" : "Elephanta Caves",
    "NP" : "Nariman Point",
    "GVP" : "Global Vipassana Pagoda",
    "KC" : "Kanheri Caves",
    "MMB" : "Mount Mary Basilica",
    "CSMVS" : "Chhatrapati Shivaji Maharaj Vastu Sangrahalaya",
    "EW" : "Essel World",
    "SSRGP" : "Sri Sri Radha Gopinath Temple",
    "JB" : "Juhu Beach"
}

code_to_url = {
    "GWOI" : "https://www.tripadvisor.in/Attraction_Review-g304554-d311667-Reviews-Gateway_of_India-Mumbai_Maharashtra.html",
    "MD" : "https://www.tripadvisor.in/Attraction_Review-g304554-d321437-Reviews-Marine_Drive-Mumbai_Maharashtra.html",
    "SHSI" : "https://www.tripadvisor.in/Attraction_Review-g304554-d622586-Reviews-Shree_Siddhivinayak-Mumbai_Maharashtra.html",
    "BWS" : "https://www.tripadvisor.in/Attraction_Review-g304554-d2704519-Reviews-Bandra_Worli_Sea_Link-Mumbai_Maharashtra.html",
    "CST" : "https://www.tripadvisor.in/Attraction_Review-g304554-d321412-Reviews-Chhatrapati_Shivaji_Terminus-Mumbai_Maharashtra.html",
    "KZM" : "https://www.tripadvisor.in/Attraction_Review-g304554-d4817230-Reviews-KidZania_Mumbai-Mumbai_Maharashtra.html",
    "MBGM" : "https://www.tripadvisor.in/Attraction_Review-g304554-d311674-Reviews-Mani_Bhavan_Gandhi_Museum-Mumbai_Maharashtra.html",
    "COL" : "https://www.tripadvisor.in/Attraction_Review-g304554-d311661-Reviews-Colaba-Mumbai_Maharashtra.html",
    "ELC" : "https://www.tripadvisor.in/AttractionProductReview-g304554-d19066869-Elephanta_Caves_Mumbai_Excursion-Mumbai_Maharashtra.html",
    "NP" : "https://www.tripadvisor.in/Attraction_Review-g304554-d3783379-Reviews-Nariman_Point-Mumbai_Maharashtra.html",
    "GVP" : "https://www.tripadvisor.in/Attraction_Review-g304554-d2514810-Reviews-Global_Vipassana_Pagoda-Mumbai_Maharashtra.html",
    "KC" : "https://www.tripadvisor.in/Attraction_Review-g304554-d1204502-Reviews-Kanheri_Caves-Mumbai_Maharashtra.html",
    "MMB" : "https://www.tripadvisor.in/Attraction_Review-g304554-d2638424-Reviews-Mount_Mary_Basilica-Mumbai_Maharashtra.html",
    "CSMVS" : "https://www.tripadvisor.in/Attraction_Review-g304554-d10071803-Reviews-Chhatrapati_Shivaji_Maharaj_Vastu_Sangrahalaya-Mumbai_Maharashtra.html",
    "EW" : "https://www.tripadvisor.in/Attraction_Review-g304554-d321430-Reviews-Essel_World-Mumbai_Maharashtra.html",
    "SSRGP" : "https://www.tripadvisor.in/Attraction_Review-g304554-d3207759-Reviews-Sri_Sri_Radha_Gopinath_Temple-Mumbai_Maharashtra.html",
    "JB" : "https://www.tripadvisor.in/Attraction_Review-g304554-d321424-Reviews-Juhu_Beach-Mumbai_Maharashtra.html"
}

list_of_destinations = ["GWOI", "SHSI", "CST"]
# list_of_destinations = ["GWOI", "MD", "SHSI", "BWS", "CST", "KZM", "MBGM", "COL", "ELC", "NP", "GVP", "KC", "MMB", "CSMVS", "EW", "SSRGP", "JB"]

if __name__ == "__main__":
    args_list = []

    for i in list_of_destinations:
        tup = (code_to_url[i], i)
        args_list.append(tup)

    processes = []
    for args in args_list:
        process = multiprocessing.Process(target=scrape_and_append_reviews, args=args)
        processes.append(process)
        process.start()

    for process in processes:
        process.join()